In [225]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
import os
from pandas.tseries.offsets import DateOffset
from scipy.signal import savgol_filter
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform, expon, reciprocal

In [226]:
# dirs
DATA_DIR = "./load.csv"
TEST_PLOT_DIR = "./test_plots/savgol_xgboost_svm_real_time/"

In [227]:
N_INPUT = 96
N_OUTPUT = 10

In [228]:
if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)
if not os.path.exists("./model"):
    os.makedirs("./model")
if not os.path.exists("./training_history"):
    os.makedirs("./training_history")

In [229]:
data = pd.read_csv(DATA_DIR)
data['Timestamp'] = pd.to_datetime(data['Timestamp'], format='%Y/%m/%d %H:%M')

In [230]:
data.describe()

,Timestamp,Load
count,35040,35040.000000
mean,2023-07-02 11:52:30,8.877887
min,2023-01-01 00:00:00,0.020000
25%,2023-04-02 05:56:15,6.380000
50%,2023-07-02 11:52:30,7.820000
75%,2023-10-01 17:48:45,10.160000
max,2023-12-31 23:45:00,24.420000
std,NaN,3.543193


In [231]:
maxValue = np.max(data['Load'])

# scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data['Load'].to_numpy().reshape(-1, 1))
data['Load'] = data_scaled

In [232]:
data

,Timestamp,Load
0,2023-01-01 00:00:00,0.445492
1,2023-01-01 00:15:00,0.427049
2,2023-01-01 00:30:00,0.445492
3,2023-01-01 00:45:00,0.420902
4,2023-01-01 01:00:00,0.422951
...,...,...
35035,2023-12-31 22:45:00,0.331148
35036,2023-12-31 23:00:00,0.270492
35037,2023-12-31 23:15:00,0.365574
35038,2023-12-31 23:30:00,0.337295


In [233]:
# Define a function to generate a list of timestamps every 2 hours within the dataset's range
def generate_timestamps(data) -> pd.DatetimeIndex:
    start = data['Timestamp'].min() + DateOffset(days=9, hours=30, minutes=0)
    end = data['Timestamp'].max() - DateOffset(hours=2, minutes=30)
    timestamps = pd.date_range(start=start, end=end, freq='2H')
    return timestamps


In [234]:
timestamps = generate_timestamps(data)
print(timestamps)

DatetimeIndex(['2023-01-11 06:00:00', '2023-01-11 08:00:00',
               '2023-01-11 10:00:00', '2023-01-11 12:00:00',
               '2023-01-11 14:00:00', '2023-01-11 16:00:00',
               '2023-01-11 18:00:00', '2023-01-11 20:00:00',
               '2023-01-11 22:00:00', '2023-01-12 00:00:00',
               ...
               '2023-12-31 02:00:00', '2023-12-31 04:00:00',
               '2023-12-31 06:00:00', '2023-12-31 08:00:00',
               '2023-12-31 10:00:00', '2023-12-31 12:00:00',
               '2023-12-31 14:00:00', '2023-12-31 16:00:00',
               '2023-12-31 18:00:00', '2023-12-31 20:00:00'],
              dtype='datetime64[ns]', length=4256, freq='2H')


In [235]:
def generate_sets_for_all_timestamps(timestamps, data):
    training_sets = []
    target_sets = []
    training_sets_time = []
    target_sets_time = []

    for timestamp in timestamps:
        # Calculate the range for the current period's data
        start_time_current = timestamp - DateOffset(days=0, hours=9, minutes=15)
        end_time_current = timestamp

        # Calculate the equivalent timestamp for last week
        timestamp_last_week = timestamp - DateOffset(days=7)
        start_time_last_week = timestamp_last_week - DateOffset(days=0, hours=9, minutes=15)
        end_time_last_week = timestamp_last_week + DateOffset(hours=2, minutes=30)

        # Calculate the target range (the next 10 steps after the current timestamp)
        target_start_time = timestamp + DateOffset(minutes=15)  # Start from the next step
        target_end_time = timestamp + DateOffset(hours=2, minutes=30)  # 10 steps

        # Filter the data for training and target sets
        current_data = data[(data['Timestamp'] >= start_time_current) & (data['Timestamp'] <= end_time_current)]
        last_week_data = data[(data['Timestamp'] >= start_time_last_week) & (data['Timestamp'] <= end_time_last_week)]
        target_data = data[(data['Timestamp'] >= target_start_time) & (data['Timestamp'] <= target_end_time)]

        # Combine current and last week data for the training set
        training_data = pd.concat([last_week_data, current_data]).reset_index(drop=True)
        
        # Save the training and target sets
        if not training_data.empty and not target_data.empty:
            training_sets.append(training_data['Load'])
            target_sets.append(target_data['Load'])
            training_sets_time.append(list(training_data['Timestamp']))
            target_sets_time.append(list(target_data['Timestamp']))

    return np.array(training_sets), np.array(target_sets), training_sets_time, target_sets_time


In [236]:
# Generate training and target sets for all the timestamps
training_sets, target_sets, training_sets_time, target_sets_time = generate_sets_for_all_timestamps(timestamps, data)

In [237]:
print(training_sets_time[0])
print(target_sets_time[0])

[Timestamp('2023-01-03 20:45:00'), Timestamp('2023-01-03 21:00:00'), Timestamp('2023-01-03 21:15:00'), Timestamp('2023-01-03 21:30:00'), Timestamp('2023-01-03 21:45:00'), Timestamp('2023-01-03 22:00:00'), Timestamp('2023-01-03 22:15:00'), Timestamp('2023-01-03 22:30:00'), Timestamp('2023-01-03 22:45:00'), Timestamp('2023-01-03 23:00:00'), Timestamp('2023-01-03 23:15:00'), Timestamp('2023-01-03 23:30:00'), Timestamp('2023-01-03 23:45:00'), Timestamp('2023-01-04 00:00:00'), Timestamp('2023-01-04 00:15:00'), Timestamp('2023-01-04 00:30:00'), Timestamp('2023-01-04 00:45:00'), Timestamp('2023-01-04 01:00:00'), Timestamp('2023-01-04 01:15:00'), Timestamp('2023-01-04 01:30:00'), Timestamp('2023-01-04 01:45:00'), Timestamp('2023-01-04 02:00:00'), Timestamp('2023-01-04 02:15:00'), Timestamp('2023-01-04 02:30:00'), Timestamp('2023-01-04 02:45:00'), Timestamp('2023-01-04 03:00:00'), Timestamp('2023-01-04 03:15:00'), Timestamp('2023-01-04 03:30:00'), Timestamp('2023-01-04 03:45:00'), Timestamp('20

In [238]:
print(training_sets[0])
print(target_sets[0])

[0.29754098 0.3147541  0.30860656 0.28196721 0.2795082  0.29016393
 0.26803279 0.29631148 0.30532787 0.29672131 0.29139344 0.3
 0.31393443 0.2807377  0.31885246 0.3        0.30737705 0.27786885
 0.30368852 0.33442623 0.28934426 0.31270492 0.31393443 0.31762295
 0.31721311 0.5647541  0.31762295 0.33237705 0.33729508 0.31352459
 0.3147541  0.32459016 0.31967213 0.33237705 0.32704918 0.34672131
 0.30737705 0.34221311 0.34959016 0.29631148 0.36516393 0.33319672
 0.35163934 0.33811475 0.35327869 0.4204918  0.37745902 0.36393443
 0.31844262 0.31106557 0.34057377 0.31393443 0.29385246 0.31844262
 0.33319672 0.32377049 0.34057377 0.31352459 0.30901639 0.31516393
 0.31967213 0.3102459  0.3147541  0.31106557 0.325      0.32827869
 0.3307377  0.32336066 0.33237705 0.33729508 0.35081967 0.33811475
 0.33565574 0.33688525 0.34467213 0.34467213 0.36393443 0.3454918
 0.35286885 0.35040984 0.34672131 0.36147541 0.35819672 0.40204918
 0.35204918 0.38032787]
[0.39713115 0.35819672 0.34344262 0.33237705 0

In [239]:
print(training_sets.shape)
print(target_sets.shape)

(4256, 86)
(4256, 10)


In [240]:
X_train, X_test, y_train, y_test = train_test_split(training_sets, target_sets, test_size=0.15, random_state=42)

In [241]:
X_train_time, X_test_time, y_train_time, y_test_time = train_test_split(training_sets_time, target_sets_time, test_size=0.15, random_state=42)

In [242]:
X_train_filtered = np.array(savgol_filter(X_train, 10, 4))
X_test_filtered = np.array(savgol_filter(X_test, 10, 4))
y_train_filtered = np.array(savgol_filter(y_train, 10, 4))
y_test_filtered = np.array(savgol_filter(y_test, 10, 4))

In [243]:
X_train_residual = X_train - X_train_filtered
X_test_residual = X_test - X_test_filtered
y_train_residual = y_train - y_train_filtered
y_test_residual = y_test - y_test_filtered

In [244]:
# param_dist = {
#     'max_depth': randint(2, 10),
#     'min_child_weight': randint(2, 10),
#     'learning_rate': uniform(0.0001, 0.1),
#     'n_estimators': randint(10, 500)
# }

# xgboost = xgb.XGBRegressor(objective='reg:squarederror')
# random_search = RandomizedSearchCV(
#     estimator=xgboost,
#     param_distributions=param_dist,
#     n_iter=200,
#     cv=3,
#     verbose=2,
#     random_state=42,
#     n_jobs=-1
# )
# random_search.fit(X_train_residual, y_train_residual)

# print("-"*86)
# print("xgboost Best parameters:", random_search.best_params_)
# print("xgboost Best score:", random_search.best_score_)
# print("-"*86)

# lr: 0.05232432600548044
# max_depth: 6
# min_child_weight: 5
# n_estimators:117

In [245]:
xgboost = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.0523,
    max_depth=6,
    min_child_weight=5,
    n_estimators=117,
)

xgboost.fit(X_train_residual, y_train_residual)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.0523, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=117, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [246]:
# param_dist = {
#     'estimator__C': reciprocal(0.0001, 0.1),
#     'estimator__epsilon': expon(scale=0.1)
# }

# svr = SVR(kernel='rbf')
# multioutput_svr = MultiOutputRegressor(svr)
# random_search = RandomizedSearchCV(estimator=multioutput_svr, param_distributions=param_dist, n_iter=1000, cv=3, verbose=2, random_state=42, n_jobs=-1)
# random_search.fit(X_train_filtered, y_train_filtered)

# print("-"*86)
# print("svm Best parameters:", random_search.best_params_)
# print("svm Best score:", random_search.best_score_)
# print("-"*86)

# svm Best parameters: {'estimator__C': 0.09959583506357932, 'estimator__epsilon': 0.004941299889430069}
# svm Best score: 0.8563983291741492


In [247]:
svr = SVR(kernel='rbf', C=0.1, epsilon=0.0049)
multioutput_svr = MultiOutputRegressor(svr)
multioutput_svr.fit(X_train_filtered, y_train_filtered)

MultiOutputRegressor(estimator=SVR(C=0.1, epsilon=0.0049))

In [248]:
pred_svr = multioutput_svr.predict(X_test_filtered)
pred_xgboost = xgboost.predict(X_test_residual)

In [252]:
y_pred = pred_svr + pred_xgboost
loss = mean_squared_error(y_test, y_pred)
print("-" * 86)
print(f'Testing Loss: {loss:.4f}')
print("-" * 86)

pred_data = scaler.inverse_transform(y_pred)
actual_data = scaler.inverse_transform(y_test)
previous_data = scaler.inverse_transform(X_test)
for i in range(actual_data.shape[0]):
    plt.figure(figsize=(12, 6))
    X1 = np.concatenate((X_test_time[i][-24:], y_test_time[i]))
    y1 = np.concatenate((previous_data[i][-24:], actual_data[i]))
    X2 = y_test_time[i]
    y_p = pred_data[i]
    y_a = actual_data[i]
    Xh = np.full(30, X1[len(X1)-10])
    yh = np.arange(0, 30, 1)
    plt.title(f"Time Series {i+1} prediction")
    plt.plot(X1, y1, '--', color='#98afc7')
    plt.plot(X2, y_p, label='Predict')
    plt.plot(X2, y_a, label='Actual')
    plt.plot(Xh, yh, color='#4863a0', alpha=0.5)
    plt.ylim(0, maxValue)
    plt.xlabel('Time step')
    plt.ylabel('Usage (kWh)')
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.savefig(TEST_PLOT_DIR+f"Time_Series_{i+1}.png")
    plt.close()

--------------------------------------------------------------------------------------
Testing Loss: 0.0033
--------------------------------------------------------------------------------------


In [253]:
from joblib import dump
# xgboost
xgboost.save_model('xgboost.model')
# svr
dump(multioutput_svr, 'svr.joblib')
# kMeans
dump(scaler, 'scaler.joblib')

/Users/elviswu/miniconda3/envs/tf/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:12:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


['scaler.joblib']